In [1]:
# Cell 1 - Install matching PyTorch + libraries
!pip install -q torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118
!pip uninstall -y safetensors
!pip install -q ultralytics opencv-python pandas numpy==1.26.4 filterpy yt-dlp scipy safetensors

Found existing installation: safetensors 0.6.2
Uninstalling safetensors-0.6.2:
  Successfully uninstalled safetensors-0.6.2


In [2]:
# Cell 2 - Download the traffic video (only if not present)
VIDEO_FILE = "traffic.mp4"
VIDEO_URL = "https://www.youtube.com/watch?v=MNn9qKG2UFI"

import os
if not os.path.exists(VIDEO_FILE):
    print("[INFO] Downloading video...")
    !yt-dlp -f mp4 "{VIDEO_URL}" -o {VIDEO_FILE}
else:
    print("[INFO] Video already present:", VIDEO_FILE)


[INFO] Downloading video...
[youtube] Extracting URL: https://www.youtube.com/watch?v=MNn9qKG2UFI
[youtube] MNn9qKG2UFI: Downloading webpage
[youtube] MNn9qKG2UFI: Downloading tv client config
[youtube] MNn9qKG2UFI: Downloading player 010fbc8d-main
[youtube] MNn9qKG2UFI: Downloading tv player API JSON
[youtube] MNn9qKG2UFI: Downloading ios player API JSON
[youtube] MNn9qKG2UFI: Downloading m3u8 information
[info] MNn9qKG2UFI: Downloading 1 format(s): 18
[download] Destination: traffic.mp4
[download] 100% of   20.91MiB in 00:00:01 at 15.66MiB/s


In [6]:
# Cell 3 - SORT implementation (embedded)
import numpy as np
from filterpy.kalman import KalmanFilter
from scipy.optimize import linear_sum_assignment as linear_assignment

def iou(bb_test, bb_gt):
    xx1 = np.maximum(bb_test[0], bb_gt[0])
    yy1 = np.maximum(bb_test[1], bb_gt[1])
    xx2 = np.minimum(bb_test[2], bb_gt[2])
    yy2 = np.minimum(bb_test[3], bb_gt[3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    # avoid division by zero
    denom = ((bb_test[2]-bb_test[0])*(bb_test[3]-bb_test[1]) +
             (bb_gt[2]-bb_gt[0])*(bb_gt[3]-bb_gt[1]) - wh)
    return wh/denom if denom>0 else 0.0

class KalmanBoxTracker(object):
    count = 0
    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array([[1,0,0,0,1,0,0],
                              [0,1,0,0,0,1,0],
                              [0,0,1,0,0,0,1],
                              [0,0,0,1,0,0,0],
                              [0,0,0,0,1,0,0],
                              [0,0,0,0,0,1,0],
                              [0,0,0,0,0,0,0]], dtype=float) # Corrected F matrix
        self.kf.H = np.array([[1,0,0,0,0,0,0],
                              [0,1,0,0,0,0,0],
                              [0,0,1,0,0,0,0],
                              [0,0,0,1,0,0,0]], dtype=float)
        self.kf.R[2:,2:] *= 10.
        self.kf.P[4:,4:] *= 1000.
        self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.01
        self.kf.Q[4:,4:] *= 0.01
        self.kf.x[:4] = bbox.reshape((4,1))
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []
        self.hits = 0
        self.hit_streak = 0
        self.age = 0

    def update(self, bbox):
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1
        self.kf.update(bbox)

    def predict(self):
        if ((self.kf.x[6] + self.kf.x[2]) <= 0):
            self.kf.x[6] *= 0.0
        self.kf.predict()
        self.age += 1
        if self.time_since_update > 0:
            self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(self.kf.x.copy())
        return self.history[-1]

    def get_state(self):
        return self.kf.x[:4].reshape((4,))

class Sort(object):
    def __init__(self, max_age=20, min_hits=2, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers = []
        self.frame_count = 0

    def update(self, dets=np.empty((0,5))):
        self.frame_count += 1
        trks = np.zeros((len(self.trackers), 5))
        to_del = []
        ret = []
        for t, trk in enumerate(self.trackers):
            pos = trk.predict()
            trks[t, :4] = pos[:4, 0] # Corrected indexing
            trks[t, 4] = 0
            if np.any(np.isnan(pos)):
                to_del.append(t)
        for t in reversed(to_del):
            self.trackers.pop(t)
        if dets.shape[0] == 0:
            dets = np.empty((0,5))
        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets, trks, self.iou_threshold)
        for m in matched:
            d, t = int(m[0]), int(m[1])
            self.trackers[t].update(dets[d, :4])
        for i in unmatched_dets:
            trk = KalmanBoxTracker(dets[i, :4])
            self.trackers.append(trk)
        i = len(self.trackers)
        for trk in reversed(self.trackers):
            d = trk.get_state()
            if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
                ret.append(np.concatenate((d, [trk.id + 1])).reshape(1, -1))
            i -= 1
            if trk.time_since_update > self.max_age:
                self.trackers.pop(i)
        if len(ret) > 0:
            return np.concatenate(ret)
        return np.empty((0,5))

def associate_detections_to_trackers(detections, trackers, iou_threshold=0.3):
    if len(trackers) == 0:
        return np.empty((0,2), dtype=int), np.arange(len(detections)), np.empty((0), dtype=int)
    iou_matrix = np.zeros((len(detections), len(trackers)), dtype=np.float32)
    for d, det in enumerate(detections):
        for t, trk in enumerate(trackers):
            iou_matrix[d, t] = iou(det, trk[:4])
    matched_indices = np.array(linear_assignment(-iou_matrix)).T if iou_matrix.size else np.empty((0,2), dtype=int)
    unmatched_detections = [d for d in range(len(detections)) if d not in matched_indices[:,0]] if matched_indices.size else list(range(len(detections)))
    unmatched_trackers = [t for t in range(len(trackers)) if t not in matched_indices[:,1]] if matched_indices.size else list(range(len(trackers)))
    matches = []
    for m in matched_indices:
        if iou_matrix[m[0], m[1]] < iou_threshold:
            unmatched_detections.append(int(m[0]))
            unmatched_trackers.append(int(m[1]))
        else:
            matches.append(m.reshape(1,2))
    if len(matches) == 0:
        matches = np.empty((0,2), dtype=int)
    else:
        matches = np.concatenate(matches, axis=0)
    return matches, np.array(unmatched_detections, dtype=int), np.array(unmatched_trackers, dtype=int)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



[INFO] Starting processing... this may take a few minutes depending on runtime GPU.


AttributeError: module 'torch' has no attribute 'uint64'

PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

[INFO] Starting processing... this may take a few minutes depending on runtime GPU.


RuntimeError: Numpy is not available

In [7]:
# Cell 4 - Main traffic flow processing using ultralytics YOLO
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from collections import defaultdict
import os

# Load YOLO model (downloads weights if necessary)
model = YOLO('yolov5s.pt')   # small model for speed; replace with a bigger one if needed

# COCO vehicle class IDs: car=2, motorcycle=3, bus=5, truck=7
VEHICLE_CLASSES = [2, 3, 5, 7]

# Define lanes (x ranges). Adjust if necessary after inspecting frame size.
LANES = {1: (0, 400), 2: (401, 800), 3: (801, 1280)}

tracker = Sort(max_age=20, min_hits=2, iou_threshold=0.3)
vehicle_lanes = {}   # track_id -> lane assigned (to avoid double counting)
counts = defaultdict(int)
records = []

cap = cv2.VideoCapture("traffic.mp4")
fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 1280)
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 720)
out = cv2.VideoWriter("annotated_output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))

frame_num = 0
print("[INFO] Starting processing... this may take a few minutes depending on runtime GPU.")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_num += 1

    detections = []
    # Ultralytics YOLO run
    results = model(frame, verbose=False)
    for r in results:
        if hasattr(r, "boxes") and r.boxes is not None:
            boxes = r.boxes.xyxy.cpu().numpy()
            confs = r.boxes.conf.cpu().numpy()
            clss = r.boxes.cls.cpu().numpy()
            for (x1, y1, x2, y2), conf, cls in zip(boxes, confs, clss):
                if int(cls) in VEHICLE_CLASSES and conf > 0.3:
                    detections.append([int(x1), int(y1), int(x2), int(y2), float(conf)])

    dets_arr = np.array(detections) if len(detections) else np.empty((0,5))
    tracks = tracker.update(dets_arr)

    # Draw lanes and counts
    for lane_id, (xmin, xmax) in LANES.items():
        cv2.line(frame, (xmin, 0), (xmin, frame_height), (255, 0, 0), 2)
        cv2.putText(frame, f"Lane {lane_id}: {counts[lane_id]}", (xmin+10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Process tracks
    for t in tracks:
        x1, y1, x2, y2, track_id = t
        x_center = int((x1 + x2) / 2)
        lane_assigned = None
        for lane_id, (xmin, xmax) in LANES.items():
            if xmin <= x_center <= xmax:
                lane_assigned = lane_id
                break
        if lane_assigned and track_id not in vehicle_lanes:
            vehicle_lanes[int(track_id)] = lane_assigned
            counts[lane_assigned] += 1
            timestamp = frame_num / fps
            records.append({"Vehicle_ID": int(track_id),
                            "Lane": lane_assigned,
                            "Frame": int(frame_num),
                            "Timestamp_sec": round(timestamp, 2)})
        # draw box and id
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"ID:{int(track_id)}", (int(x1), int(y1)-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
pd.DataFrame(records).to_csv("vehicle_counts.csv", index=False)
print("[INFO] Finished. Saved: annotated_output.mp4, vehicle_counts.csv")
print("[INFO] Final counts:", dict(counts))

PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

[INFO] Starting processing... this may take a few minutes depending on runtime GPU.
[INFO] Finished. Saved: annotated_output.mp4, vehicle_counts.csv
[INFO] Final counts: {1: 1114, 2: 57, 3: 0}


In [8]:
# Cell 5 - Download outputs to your local machine
from google.colab import files
files.download("annotated_output.mp4")
files.download("vehicle_counts.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Restart runtime
import os
os.kill(os.getpid(), 9)